# A simple micro-strip antenna in frequency domain

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

construction in mm, scale to m later:

In [ ]:
height = 2
wp = WorkPlane ( Axes ((0,0,height), Z, X))
striptop = wp.MoveTo(0.5, -0.5).Line(100).Rotate(-90).Line(10).Rotate(90).Line(20).Rotate(90).Line(21) \
        .Rotate(90).Line(20).Rotate(90).Line(10).Rotate(-90).Line(100).Close().Face()
striptop.faces.name="strip"

stripbot = striptop.Move(-2*height*Z)

port = WorkPlane( Axes((0.5,0,0), -X,Z)).RectangleC(2*height, 1).Face()
port.faces.name="port"
strip = Glue([striptop, stripbot])
shape = Glue([strip, port])

shape.faces["strip"].col=(1,0,0)
shape.faces["port"].col=(1,1,0)

In [ ]:
Draw (strip);

In [ ]:
mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=5))
Draw (mesh);

In [ ]:
#fes = HCurl(mesh, order=1, complex=True, dirichlet="strip")
#print ("ndof = ", fes.ndof)
#u,v = fes.TnT()

In [ ]:
fesHCurl = HCurl(mesh, order=1, complex=True, dirichlet="strip")
fesHDiv = HDivSurface(mesh, order=1, complex = True, dirichlet="port")

uHCurl,vHCurl = fesHCurl.TnT()
uHDiv, vHDiv = fesHDiv.TnT()

In [ ]:
mu0 = 1.257e-6  * 0.001
eps0 = 1.247e-12 * 0.001
f = 20e9
omega = 2*pi*f
kappa = omega*sqrt( eps0 * mu0 )

In [ ]:
m = GridFunction(fesHCurl)
j = GridFunction(fesHDiv)

# boundary conditions:
# Dirichlet Data m_s = 0 on strip 
# Neumann Data j_p = given on port

# Ansatz
#             E(x) = S_E^(on port)( j_p ) + S_E^(on strip) ( j_s ) + S_M^(on port) (m_p) + S_M^(on strip) (m_s) 
# m_s = 0 ==> E(x) = S_E^(on port)( j_p ) + S_E^(on strip) ( j_s ) + S_M^(on port) (m_p)

# g_R == 0, i.e., extension of homogeneous Dirichlet data on stripe (m_s=0) to whole boundary (shape) 
# g_N extension of given Neumann data j_p to whole boundary (shape)

# Coupled System
#  on strip:  V j_s + K m_p = (1/2 M  - K) g_R - V g_N      ===>  V j_s + K m_p = V g_N
#  on port:  Kt j_s + D m_p = (1/2 Mt - Kt) g_N - D g_R     ===> Kt j_s + D m_p = (1/2 Mt - Kt) g_N 

with TaskManager():
    V = MaxwellSingleLayerPotentialOperator(fesHDiv, definedon=mesh.Boundaries("strip"), 
                                            intorder=12, leafsize=40, eta=3., eps=1e-4, method="aca", testhmatrix=False)
    K = MaxwellDoubleLayerPotentialOperator(fesHCurl, fesHDiv,  
                                     trial_definedon=mesh.Boundaries("port"),  test_definedon=mesh.Boundaries("strip"), 
                                     intorder=12, leafsize=40, eta=3., eps=1e-4, method="aca", testhmatrix=False)
    D = MaxwellHypersingularOperator(fesHCurl, fesHDiv, definedon=mesh.Boundaries("port"), test_definedon=mesh.Boundaries("port"),  
                                     intorder=12, leafsize=40, eta=3., eps=1e-4, method="aca", testhmatrix=False) # wrong spaces ?
    Kt = MaxwellDoubleLayerPotentialOperator(fesHCurl, fesHDiv,  
                                     trial_definedon=mesh.Boundaries("port"),  test_definedon=mesh.Boundaries("strip"), 
                                     intorder=12, leafsize=40, eta=3., eps=1e-4, method="aca", testhmatrix=False) # use its transposed ?
    M = BilinearForm( uHCurl.Trace() * vHDiv.Trace()* ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv>  # definedon missing

In [ ]:
# Neumann Daten auf port, 
# homogene Dirichlet Daten auf strip

pre = BilinearForm( uHDiv.Trace() * vHDiv.Trace() *ds ).Assemble().mat.Inverse(freedofs=fesHDiv.FreeDofs()) 


In [ ]:
gfu = GridFunction(fes)
inv = a.mat.Inverse(fes.FreeDofs())
f = LinearForm( CF( (0,0,jz)) * v.Trace() * ds("port")).Assemble() # v - Test Funktion von H(curl) Projektion des port current -> Freiheitsgrade gesetzt in Hdiv Surface
gfu.vec.data = inv * f.vec

in gui, switch on "Clipping Plane" for drawing field in clipping plane:

In [ ]:
#Draw (gfu[2], mesh, clipping = { "z" : 1}, animate_complex=True );